In [1]:
import sys
sys.path.remove('/home/sentic/.local/lib/python3.6/site-packages')

import torch
# torch.backends.cudnn.benchmark=True

torch.cuda.set_enabled_lms(True)
torch.cuda.set_size_lms(120009999)

device_id = 0
torch.cuda.set_device(device_id)

#root = "../train"
root = "/home/sentic/MICCAI/data/train/"
use_gpu = True
n_epochs = 200
batch_size = 1
use_amp = False
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import LambdaLR

from torch.utils.data import DataLoader

from model import LargeCascadedModel
from dataset import BraTS
from losses import DiceLoss
from tqdm import tqdm_notebook, tqdm
from learning_rate import GradualWarmupScheduler, PolyLR

import pytorch_warmup as warmup

In [2]:
path_resume = "./checkpoints/checkpoint_188.pt"
checkpoint_optimizer = True
model = LargeCascadedModel(inplanes_encoder_1=4, channels_encoder_1=16, num_classes_1=3,
                           inplanes_encoder_2=7, channels_encoder_2=32, num_classes_2=3)

if use_gpu:
    model = model.to("cuda")

start_lr = 1e-4
optimizer = optim.Adam(list(filter(lambda p: p.requires_grad, model.parameters())), lr=start_lr, weight_decay=1e-5)
scheduler = PolyLR(optimizer, max_decay_steps=n_epochs, end_learning_rate=1e-8, power=0.9)

if path_resume is not None:
    dict_state = torch.load(path_resume)
    model.load_state_dict(dict_state['state_dict'])
    if checkpoint_optimizer:
        optimizer.load_state_dict(dict_state['optimizer'])
        last_epoch = dict_state['epoch']
    else:
        last_epoch = 0
else:
    last_epoch = 0
#scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.85)


# warm_up = GradualWarmupScheduler(optimizer, multiplier=2, total_epoch=5, after_scheduler=scheduler)
# warm_up = warmup.LinearWarmup(optimizer, warmup_period=369*5)

BraTSDataset = BraTS(root=root, phase="train", desired_depth=128, desired_height=240, desired_width=240, 
                     normalize_flag=True, scale_intensity_flag=True, shift_intesity_flag=True, flip_axes_flag=True)
dataloader = DataLoader(BraTSDataset, batch_size=batch_size, shuffle=True)

diceLoss = DiceLoss()

In [ ]:
string = ""
for epoch in range(n_epochs - last_epoch):
    epoch_loss = 0
    for param_group in optimizer.param_groups:
        print("Learning rate =", param_group['lr'])
        break
    for ix, loader in tqdm(enumerate(dataloader)):
        volume, mask, contour, patient = loader
        torch.cuda.empty_cache()
        if use_gpu:
            volume = volume.to("cuda")
            mask = mask.to("cuda")
            contour = contour.to("cuda")
            
        
        optimizer.zero_grad()
        decoded_region1, decoded_region2, decoded_region3, decoded_contour = model(volume)
        
        loss_dice_region1 = diceLoss(decoded_region1, mask)
        loss_dice_region2 = diceLoss(decoded_region2, mask)
        loss_dice_region3 = diceLoss(decoded_region3, mask)
        loss_dice_contour = diceLoss(decoded_contour, contour)
        loss = loss_dice_region1 + loss_dice_region2 + loss_dice_region3 + loss_dice_contour
        
        with torch.no_grad():
            string += "P: {} DC R1: {}, DC R2: {}, DC R3: {},DC C: {}".format(patient, - np.round(loss_dice_region1.item(), 4),
                                                                              - np.round(loss_dice_region2.item(), 4),
                                                                              - np.round(loss_dice_region3.item(), 4),
                                                                               - np.round(loss_dice_contour.item(), 4))
            print("P: {} DC R1: {}, DC R2: {}, DC R3: {}, DC C: {}".format(patient, - np.round(loss_dice_region1.item(), 4),
                                                                           - np.round(loss_dice_region2.item(), 4),
                                                                           - np.round(loss_dice_region3.item(), 4),
                                                                           - np.round(loss_dice_contour.item(), 4)))


            string += "\n"
            
        with torch.no_grad():
            epoch_loss += loss_dice_region3.item()
        
        loss.backward()
        optimizer.step()
        
        scheduler.step(step=(epoch + last_epoch))
        # warm_up.dampen()
        
        
        del volume
        del mask
        del contour
        del decoded_region1
        del decoded_region2
        del decoded_region3
        del decoded_contour
        del loss
        del loss_dice_region1
        del loss_dice_region2
        del loss_dice_region3
        del loss_dice_contour
          
    epoch_loss = epoch_loss / (ix + 1)
    string += "Epoch {}: loss {}".format(epoch, epoch_loss)
    print("Epoch {}: loss {}".format(epoch, epoch_loss))
    for param_group in optimizer.param_groups:
        print("Learning rate =", param_group['lr'])
        break
    string += "\n"
    if epoch % 10 == 0 and epoch != 0:
        with open("results.txt", "w") as fhandle:
            print("Logged the results")
            fhandle.write(string)
    if (epoch + last_epoch) % 1 == 0 and epoch != 0:
        path_checkpoint = "/home/sentic/MICCAI/Madu/APPROACH_6/checkpoints/checkpoint_" + str(epoch + last_epoch) + ".pt"
        dict_state = {'epoch': epoch + last_epoch,
                     'state_dict': model.state_dict(),
                     'optimizer': optimizer.state_dict()
                     }
        print(path_checkpoint)
        torch.save(dict_state, path_checkpoint)

0it [00:00, ?it/s]

Learning rate = 7.958637544298869e-06
P: ('BraTS20_Training_148',) DC R1: 0.8937, DC R2: 0.9039, DC R3: 0.9132, DC C: 0.6045


1it [00:22, 22.53s/it]

P: ('BraTS20_Training_239',) DC R1: 0.9209, DC R2: 0.9287, DC R3: 0.944, DC C: 0.7322


2it [00:42, 21.63s/it]

...

P: ('BraTS20_Training_041',) DC R1: 0.9173, DC R2: 0.9186, DC R3: 0.9261, DC C: 0.6875


78it [25:50, 19.80s/it]

P: ('BraTS20_Training_130',) DC R1: 0.9442, DC R2: 0.9477, DC R3: 0.9494, DC C: 0.676


79it [26:10, 19.79s/it]

P: ('BraTS20_Training_358',) DC R1: 0.9634, DC R2: 0.9668, DC R3: 0.9734, DC C: 0.8055


In [ ]:
# path_checkpoint = "/home/sentic/MICCAI/Madu/APPROACH_6/checkpoints/checkpoint_" + str(42) + ".pt"
# dict_state = {'epoch': 42,
#              'state_dict': model.state_dict(),
#              'optimizer': optimizer.state_dict()
#              }
# print(path_checkpoint)
# torch.save(dict_state, path_checkpoint)

In [ ]:
for param_group in optimizer.param_groups:
    print("Learning rate =", param_group['lr'])
    break